In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import json
import torch
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
model = GPT2LMHeadModel.from_pretrained(model_name)


2024-04-19 16:18:53.540785: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-19 16:18:53.540914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-19 16:18:53.670481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
# Load prompt and response data from JSON file
from sklearn.model_selection import train_test_split

file_path = "/kaggle/input/commands/dataset.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)


# Extract prompts and commands
prompts = [item['prompt'] for item in data]
commands = [item['command'] for item in data]

# Create a dictionary
data_dict = {
    'prompt': prompts,
    'command': commands
}

# train, test = train_test_split(data,test_size=0.15)
# print(train)

# train_dataset = Dataset.from_dict(train)
# test_dataset = Dataset.from_dict(test)
# print(train_dataset)

# Create Dataset object
dataset = Dataset.from_dict(data_dict)
print(dataset[0])


{'prompt': 'List files and directories in the current directory', 'command': 'ls'}


In [5]:
# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["prompt"], example["command"], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/179 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'command', 'input_ids', 'attention_mask'],
    num_rows: 179
})

In [6]:
# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [7]:
# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./finetuned_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=800,
    save_total_limit=2,
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./finetuned_gpt2")


# Save the tokenizer
tokenizer.save_pretrained("./finetuned_gpt2")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


('./finetuned_gpt2/tokenizer_config.json',
 './finetuned_gpt2/special_tokens_map.json',
 './finetuned_gpt2/vocab.json',
 './finetuned_gpt2/merges.txt',
 './finetuned_gpt2/added_tokens.json')

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned GPT-2 model and tokenizer
model_name = "./finetuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [10]:
# Define the prompt
prompt = "create a directory named 'test'"

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Generate response
output = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

In [25]:
print(output[0])

tensor([27914,   262,  2393,  3706,   705, 20688,    13, 14116,     6, 26224,
         1672,    13, 14116,  1672,    13, 14116,   930,   266,    66,   532,
           75,  1672,    13, 14116,   930,  7894,   532,    77,   352,   930,
         7894,   532,    77,   352,   930,   266,    66,   532,    75,  1672,
           13, 14116,   930,  7894,   532,    77,   352,   930,  7894,   532])


In [11]:
# Decode the generated response


response = tokenizer.decode(output[0], skip_special_tokens=True)

# # Print the response
print(response)

create a directory named 'test'mkdir test_directory test_test_test_test_test_test_test_test_test_test_test_test_test_test_test_test_test_test_test_
